In [142]:
import pandas as pd
from selenium import webdriver
import time
import datetime

def list_dates(start,end):
    """ This creates a list of of dates between the 'start' date and the 'end' date """
    # create datetime object for the start and end dates
    start = datetime.datetime.strptime(start, '%Y-%m-%d')
    end = datetime.datetime.strptime(end, '%Y-%m-%d')
    # generates list of dates between start and end dates
    step = datetime.timedelta(days=1)
    dates = []
    while start <= end:
        dates.append(start.date())
        start += step
    # return the list of dates in string format
    return [str(date) for date in dates]

start='2018-01-01'
end='2018-12-31'

url='https://www.wunderground.com/history/daily/us/fl/orlando/date/'
data=[]

for date in list_dates(start,end):
    driver = webdriver.Chrome('/usr/local/bin/chromedriver')
    driver.get(url+str(date))
    time.sleep(2)

    from bs4 import BeautifulSoup
    soup = BeautifulSoup(driver.page_source)
    table=soup.find_all('table')[-1]

    #Table Headers
    table_headers = table.find('thead')
    headers = table_headers.find_all('th')
    header=["Date"]+[ele.text.strip() for ele in headers]

    #Table Body
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([date]+[ele for ele in cols if ele])
    
    driver.close()
    
df=pd.DataFrame(data,columns=header)
df["Temp_(F)"]=df["Temperature"].str.rstrip(' \nF').astype(int)
df["Dew_Point_(F)"]=df["Dew Point"].str.rstrip(' \nF').astype(int)
df["Humidity_(%)"]=df["Humidity"].str.rstrip(' \n%').astype(int)
df["Wind_Speed_(mph)"]=df["Wind Speed"].str.rstrip(' \nmph').astype(int)
df["Wind_Gust_(mph)"]=df["Wind Gust"].str.rstrip(' \nmph')
df["Pressure_(in)"]=df["Pressure"].str.rstrip(' \nin')
df["Precip_(in)"]=df["Precip."].str.rstrip(' \nin')

In [143]:
df_cleaned=df[["Date","Time","Temp_(F)","Dew_Point_(F)","Humidity_(%)","Wind","Wind_Speed_(mph)","Wind_Gust_(mph)","Pressure_(in)","Precip_(in)","Condition"]]
df_cleaned

,Date,Time,Temp_(F),Dew_Point_(F),Humidity_(%),Wind,Wind_Speed_(mph),Wind_Gust_(mph),Pressure_(in),Precip_(in),Condition
0,2018-12-01,1:53 AM,67,59,76,ESE,6,0,29.95,0.0,Cloudy
1,2018-12-01,2:53 AM,66,59,78,ESE,3,0,29.94,0.0,Cloudy
2,2018-12-01,3:53 AM,65,60,84,E,5,0,29.92,0.0,Cloudy
3,2018-12-01,4:53 AM,65,60,84,SE,5,0,29.91,0.0,Cloudy
4,2018-12-01,5:53 AM,65,61,87,E,3,0,29.91,0.0,Cloudy
...,...,...,...,...,...,...,...,...,...,...,...
927,2018-12-31,8:53 PM,71,62,73,SSE,6,0,30.04,0.0,Fair
928,2018-12-31,9:53 PM,72,64,76,ESE,7,0,30.05,0.0,Mostly Cloudy
929,2018-12-31,10:53 PM,72,68,87,SE,5,0,30.06,0.0,Mostly Cloudy
930,2018-12-31,11:53 PM,71,68,90,SSE,6,0,30.06,0.0,Cloudy


In [19]:
df_cleaned.to_pickle("May-18-2.pickle")

In [23]:
df_May_18_1=pd.read_pickle("May-18-1.pickle")

In [24]:
df_May_18_2=pd.read_pickle("May-18-2.pickle")

In [28]:
df_May_18=pd.concat([df_May_18_1,df_May_18_2])

In [31]:
df_May_18.reset_index(inplace=True, drop=True)

In [32]:
df_May_18

,Date,Time,Temp_(F),Dew_Point_(F),Humidity_(%),Wind,Wind_Speed_(mph),Wind_Gust_(mph),Pressure_(in),Precip_(in),Condition
0,2018-05-01,12:53 AM,65,56,73,ENE,8,0,30.10,0.0,Fair
1,2018-05-01,1:53 AM,65,55,70,ENE,8,0,30.08,0.0,Partly Cloudy
2,2018-05-01,2:53 AM,63,55,75,NNE,7,0,30.07,0.0,Fair
3,2018-05-01,3:53 AM,65,54,68,NNE,8,0,30.07,0.0,Fair
4,2018-05-01,4:53 AM,62,54,75,N,6,0,30.07,0.0,Fair
...,...,...,...,...,...,...,...,...,...,...,...
912,2018-05-31,9:53 PM,73,73,100,SSE,8,0,29.92,0.0,Cloudy
913,2018-05-31,10:53 PM,72,72,100,S,5,0,29.91,0.0,Mostly Cloudy
914,2018-05-31,11:08 PM,72,72,100,SSW,6,0,29.92,0.0,Cloudy
915,2018-05-31,11:19 PM,72,72,100,SSW,3,0,29.92,0.0,Cloudy


In [144]:
df_cleaned.to_pickle("Dec-18.pickle")

In [58]:
df_June_18=pd.read_pickle("June-18.pickle")

In [53]:
df_July_18_1=pd.read_pickle("July-18-1.pickle")

In [54]:
df_July_18_2=pd.read_pickle("July-18-2.pickle")

In [56]:
df_Aug_18_1=pd.read_pickle("Aug-18-1.pickle")

In [57]:
df_Aug_18_2=pd.read_pickle("Aug-18-2.pickle")

In [59]:
df_May_Aug_18=pd.concat([df_May_18_1,df_May_18_2,df_June_18,df_July_18_1,df_July_18_2,df_Aug_18_1,df_Aug_18_2])

In [60]:
df_May_Aug_18.reset_index(inplace=True, drop=True)

In [75]:
df_May_Aug_2018=pd.concat([df_May_18_1,df_May_18_2,df_June_18,df_July_18_1,df_July_18_2,df_Aug_18_1,df_Aug_18_2])

In [76]:
df_May_Aug_2018.reset_index(inplace=True, drop=True)

In [77]:
df_May_Aug_2018

,Date,Time,Temp_(F),Dew_Point_(F),Humidity_(%),Wind,Wind_Speed_(mph),Wind_Gust_(mph),Pressure_(in),Precip_(in),Condition
0,2018-05-01,12:53 AM,65,56,73,ENE,8,0,30.10,0.0,Fair
1,2018-05-01,1:53 AM,65,55,70,ENE,8,0,30.08,0.0,Partly Cloudy
2,2018-05-01,2:53 AM,63,55,75,NNE,7,0,30.07,0.0,Fair
3,2018-05-01,3:53 AM,65,54,68,NNE,8,0,30.07,0.0,Fair
4,2018-05-01,4:53 AM,62,54,75,N,6,0,30.07,0.0,Fair
...,...,...,...,...,...,...,...,...,...,...,...
3581,2018-08-31,7:53 PM,83,74,74,E,8,0,29.93,0.0,Mostly Cloudy
3582,2018-08-31,8:53 PM,81,75,82,E,7,0,29.95,0.0,Mostly Cloudy
3583,2018-08-31,9:53 PM,80,75,85,ENE,8,0,29.96,0.0,Mostly Cloudy
3584,2018-08-31,10:53 PM,80,76,87,E,5,0,29.99,0.0,Mostly Cloudy


In [66]:
df_May_Aug_18['Date'] = pd.to_datetime(df_May_Aug_18['Date'])

In [67]:
df_May_Aug_18['Time'] = pd.to_datetime(df_May_Aug_18['Time'])

In [71]:
df_May_Aug_18['Pressure_(in)']=df_May_Aug_18['Pressure_(in)'].astype(float)

In [85]:
df_May_Aug_18.to_csv("Weather_May_Aug_18_test.csv")

In [80]:
df_May_Aug_2018['Date_Time'] = df_May_Aug_2018['Date']+' '+df_May_Aug_2018['Time']

In [83]:
df_May_Aug_2018['Date_Time'] = pd.to_datetime(df_May_Aug_2018['Date_Time'])

In [101]:
df_May_Aug_2018.to_csv("Weather_May_Aug_2018.csv")

In [102]:
Weather_May_Aug2018=pd.read_csv('Weather_May_Aug_2018.csv')

In [112]:
Weather_May_Aug_2018['Date'] = Weather_May_June_2018['Date'].dt.strftime('%Y-%m-%d')

In [111]:
Weather_May_Aug_2018['Date'] = pd.to_datetime(Weather_May_June_2018['Date'])

In [113]:
Weather_May_Aug_2018['Date_Time'] = pd.to_datetime(Weather_May_June_2018['Date_Time'])

In [114]:
Weather_May_Aug_2018['Hour'] = Weather_May_June_2018['Date_Time'].dt.hour

In [115]:
Weather_May_Aug_2018['Date_and_Hour']=Weather_May_June_2018['Date'].map(str)+'_'+Weather_May_June_2018['Hour'].map(str)

In [118]:
Weather_May_Aug_2018.to_csv('WeatherU_May_Aug_2018.csv')

In [119]:
Weather_May_Aug_2018

,Date,Time,Temp_(F),Dew_Point_(F),Humidity_(%),Wind,Wind_Speed_(mph),Wind_Gust_(mph),Pressure_(in),Precip_(in),Condition,Date_Time,Hour,Date_and_Hour
0,2018-05-01,12:53 AM,65,56,73,ENE,8,0,30.10,0.0,Fair,2018-05-01 00:53:00,0,2018-05-01_0
1,2018-05-01,1:53 AM,65,55,70,ENE,8,0,30.08,0.0,Partly Cloudy,2018-05-01 01:53:00,1,2018-05-01_1
2,2018-05-01,2:53 AM,63,55,75,NNE,7,0,30.07,0.0,Fair,2018-05-01 02:53:00,2,2018-05-01_2
3,2018-05-01,3:53 AM,65,54,68,NNE,8,0,30.07,0.0,Fair,2018-05-01 03:53:00,3,2018-05-01_3
4,2018-05-01,4:53 AM,62,54,75,N,6,0,30.07,0.0,Fair,2018-05-01 04:53:00,4,2018-05-01_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3581,2018-08-31,7:53 PM,83,74,74,E,8,0,29.93,0.0,Mostly Cloudy,2018-08-31 19:53:00,19,2018-08-31_19
3582,2018-08-31,8:53 PM,81,75,82,E,7,0,29.95,0.0,Mostly Cloudy,2018-08-31 20:53:00,20,2018-08-31_20
3583,2018-08-31,9:53 PM,80,75,85,ENE,8,0,29.96,0.0,Mostly Cloudy,2018-08-31 21:53:00,21,2018-08-31_21
3584,2018-08-31,10:53 PM,80,76,87,E,5,0,29.99,0.0,Mostly Cloudy,2018-08-31 22:53:00,22,2018-08-31_22
